<a href="https://colab.research.google.com/github/BiancadeFrancisco/BigData_DataQuality2/blob/main/C%C3%B3pia_de_DATAQUALITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar a vesão 3.0.3 do PySpark
!pip install pyspark==3.0.3

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.1/209.1 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 18.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435950 sha256=4301dab9f6e0760a50c7a8e42a43b5a63b648e02c59b297446254d06d2e4ca44
  Stored in directory: /root/.cache/pip/wheels/40/50/14/79047c3c171b701e591d287b78a201214d9c8e0b93fef64458
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://b46e-35-194-143-42.ngrok-free.app


In [ ]:
!pip install pydeequ

In [ ]:
from pyspark.sql import SparkSession


In [ ]:
!python -- version

python3: can't open file '/content/version': [Errno 2] No such file or directory


In [ ]:
import os

# Defina a versão do Spark que você deseja usar
os.environ['SPARK_VERSION'] = '3.0'

# Agora, importe o PyDeequ
from pydeequ import *

In [ ]:
import pydeequ

In [ ]:
spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .config("spark.jars.packages", pydeequ.deequ_maven_coord)
      .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
      .appName("SparkSQL")
      .getOrCreate()
)


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType
from pyspark.sql.functions import col

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = '/content/case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

df = df.withColumn(
      'destinatario_nome', col('destinatario').getField('nome')
    ).withColumn(
      'destinatario_banco', col('destinatario').getField('banco')
    ).withColumn(
      'destinatario_tipo', col('destinatario').getField('tipo')
    ).withColumn(
      'remetente_nome', col('remetente').getField('nome')
    ).withColumn(
      'remetente_banco', col('remetente').getField('banco')
    ).withColumn(
      'remetente_tipo', col('remetente').getField('tipo')
).drop('remetente', 'destinatario')

In [ ]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, ApproxCountDistinct, Completeness, Compliance, Mean, Size


analysisResult = (
    AnalysisRunner(spark).onData(df)
    .addAnalyzer(Size()) # análise para verificar o tamanho do arquivo
    .addAnalyzer(Completeness('id_transacao')) # análise para verificar a completude da coluna id_transacao
    .addAnalyzer(Compliance("valor", "valor > 0")) # vai analisar se a coluna valor atende o querisito imposto: > 0
    .run()
)


In [ ]:
analysisResult

JavaObject id=o138

In [ ]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)

In [ ]:
analysisResult_df.show()

# resultados: temos 100 mil linhas no dataset, a coluna id está completa com 100% dos valores e na coluna valor ele tem 99,97% de chance dos números serem maiores que 0.

+-------+------------+------------+--------+
| entity|    instance|        name|   value|
+-------+------------+------------+--------+
|Dataset|           *|        Size|100000.0|
| Column|id_transacao|Completeness|     1.0|
| Column|       valor|  Compliance| 0.99972|
+-------+------------+------------+--------+



In [ ]:
# CASO EU QUEIRA UMA SUGESTÃO DO PYDEEQU SOBRE ALGUMAS ANÁLISES QUE SERIAM INTERESSANTES EU FAZER COM OS DADOS DO MEU DATASET:

from pydeequ.suggestions import ConstraintSuggestionRunner, DEFAULT

suggestionResult = ConstraintSuggestionRunner(spark).onData(df).addConstraintRule(DEFAULT()).run()



In [ ]:
# FUNÇÃO PARA QUE ELE MESMO LISTE AS SUGESTÕES DE ANÁLISES E OS CÓDIGOS DELES

for sugg in suggestionResult['constraint_suggestions']:
  print(f"Sugestao de Constraint: \'{sugg['column_name']}\': {sugg['description']}")
  print(f"PySpark Code: {sugg['code_for_constraint']}\n")

Sugestao de Constraint: 'destinatario_nome': 'destinatario_nome' is not null
PySpark Code: .isComplete("destinatario_nome")

Sugestao de Constraint: 'remetente_nome': 'remetente_nome' has value range 'Jonathan Gonsalves'
PySpark Code: .isContainedIn("remetente_nome", ["Jonathan Gonsalves"])

Sugestao de Constraint: 'remetente_nome': 'remetente_nome' is not null
PySpark Code: .isComplete("remetente_nome")

Sugestao de Constraint: 'id_transacao': 'id_transacao' is not null
PySpark Code: .isComplete("id_transacao")

Sugestao de Constraint: 'id_transacao': 'id_transacao' has no negative values
PySpark Code: .isNonNegative("id_transacao")

Sugestao de Constraint: 'id_transacao': 'id_transacao' is unique
PySpark Code: .isUnique("id_transacao")

Sugestao de Constraint: 'remetente_banco': 'remetente_banco' has value range 'BTG'
PySpark Code: .isContainedIn("remetente_banco", ["BTG"])

Sugestao de Constraint: 'remetente_banco': 'remetente_banco' is not null
PySpark Code: .isComplete("remetente_

In [ ]:
from pydeequ.checks import Check, CheckLevel, ConstrainableDataTypes
from pydeequ.verification import VerificationResult, VerificationSuite

check = Check(spark, CheckLevel.Warning, "Review Check")
error = Check(spark, CheckLevel.Error, "Error")

In [ ]:
checkResult = (
    VerificationSuite(spark)
      .onData(df)
      .addCheck(
        check.hasDataType("id_transacao", ConstrainableDataTypes.Integral)
        .isNonNegative("id_transacao")
        .isComplete("id_transacao")
      )
  .run()
)

In [ ]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check       |check_level|check_status|constraint                                                                                                                  |constraint_status|constraint_message|
+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Review Check|Warning    |Success     |AnalysisBasedConstraint(DataType(id_transacao,None),<function1>,Some(<function1>),None)                                     |Success          |                  |
|Review Check|Warning    |Success     |ComplianceConstraint(Compliance(id_transacao is non-negative,COALESCE(CAST(id_transacao AS DECIMAL(20,10)), 0.0) >= 0,None))|Success          |          

In [ ]:
# testar um erro para visualizar como o modelo responde:


checkResult = (
    VerificationSuite(spark)
      .onData(df)
      .addCheck(
        error
          .isContainedIn("remetente_tipo", ["CNPJ"])
      )
  .run()
)

In [ ]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|check|check_level|check_status|constraint                                                                                                                      |constraint_status|constraint_message                                  |
+-----+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|Error|Error      |Error       |ComplianceConstraint(Compliance(remetente_tipo contained in CNPJ,`remetente_tipo` IS NULL OR `remetente_tipo` IN ('CNPJ'),None))|Failure          |Value: 0.0 does not meet the constraint requirement!|
+-----+-----------+------------+------------------------------------

In [ ]:
# ESSE ASSUNTO É MUITO RELEVANTE E ESTÁ EM ALTA, VERIFICAR DOCUMENTAÇÃO E OUTROS EXEMPLOS E PARÂMETROS.